## Masters of the Great Web. ERC721 Analysis

In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
from IPython.display import HTML, display
import pandas as pd

from config import PROJECT_ID, INITIAL_TS, SNAPSHOT_TS, \
    ERC721_ANALYSIS_DATASET_NAME, ERC721_AMOUNT_TABLE_NAME, ERC721_ANALYSIS_DISTRIBUTION_TABLE_NAME
from utils import drop_table, create_table

DROP_TABLES = True
CREATE_TABLES = True

### Distribution Rule

<table style="text-align: center">
    <thead style="text-align: center">
        <tr>
            <th rowspan=2></th>
            <th colspan=3>Grade</th>
        </tr>
        <tr>
            <th>1</th>
            <th>2</th>
            <th>3</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td style="text-align: left"> Amount of ERC721 tokens </td>
            <td> > 0 NFT </td>
            <td> > 10 NFT </td>
            <td> > 100 NFT </td>
        </tr>
    </tbody>
</table>

In [2]:
min_number_of_tokens = 0
boundary_erc721_amount = [0, 10, 100]

### Set Token Data

In [3]:
erc721_tokens_dict = {
                'Gitcoin Kudos': '0x2aea4add166ebf38b63d09a75de1a7b94aa24163',
                'ENS': '0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85',
                'LAND': '0xf87e31492faf9a91b02ee0deaad50d51d56d5d4d',
                'Unicorns': '0x89205a3a3b2a69de6dbf7f01ed13b2108b2c43e7',
                'DRAGON': '0x960f401aed58668ef476ef02b2a2d43b83c261d8',
                'Cryptopunks': '0xb47e3cd837ddf8e4c57f05d70ab865de6e193bbb'}
erc721_tokens = {k: v.lower() for k, v in erc721_tokens_dict.items()}
erc721_tokens_address_tuple = tuple(erc721_tokens_dict.values())

### Get Balances

In [4]:
query_1 = f'''
        WITH token_transfers AS (
                SELECT
                    token_address,
                    address,
                    sum(value) as amount
                FROM (
                    SELECT
                        token_address,
                        from_address as address,
                        - count(value) as value
                    FROM `bigquery-public-data.crypto_ethereum.token_transfers`
                    WHERE token_address IN {erc721_tokens_address_tuple}
                      AND from_address NOT IN ('0x0000000000000000000000000000000000000000')
                      AND block_timestamp >= '{INITIAL_TS}'
                      AND block_timestamp <= '{SNAPSHOT_TS}'
                    GROUP BY token_address, from_address
                    UNION ALL
                    SELECT
                        token_address,
                        to_address as address,
                        count(value) as value
                    FROM `bigquery-public-data.crypto_ethereum.token_transfers`
                    WHERE token_address IN {erc721_tokens_address_tuple}
                      AND to_address NOT IN ('0x0000000000000000000000000000000000000000')
                      AND block_timestamp >= '{INITIAL_TS}'
                      AND block_timestamp <= '{SNAPSHOT_TS}'
                    GROUP BY token_address, to_address
                    )
                GROUP BY token_address, address
        )
        SELECT
            token_address,
            address as address,
            amount
        FROM token_transfers
'''

if DROP_TABLES:
    drop_table(ERC721_AMOUNT_TABLE_NAME, dataset_name=ERC721_ANALYSIS_DATASET_NAME)

if CREATE_TABLES:
    create_table(query_1, ERC721_AMOUNT_TABLE_NAME, dataset_name=ERC721_ANALYSIS_DATASET_NAME)

Table erc721_analysis:erc721_amounts has been deleted.
Table erc721_analysis:erc721_amounts has been created and filled 59346 rows.


### Create Distribution Table

In [5]:
query_2 = f'''
SELECT
    address,
    CASE
        WHEN sum_amount > {boundary_erc721_amount[2]} THEN 3
        WHEN sum_amount > {boundary_erc721_amount[1]} THEN 2
        WHEN sum_amount > {boundary_erc721_amount[0]} THEN 1
        ELSE null
    END
    AS grade,
    sum_amount,
    number_of_tokens
FROM (
    SELECT
        address,
        count( distinct token_address) as number_of_tokens,
        sum(amount) as sum_amount
    FROM `{PROJECT_ID}.{ERC721_ANALYSIS_DATASET_NAME}.{ERC721_AMOUNT_TABLE_NAME}`
    WHERE amount > 0
    GROUP BY address)
WHERE number_of_tokens > {min_number_of_tokens}
'''

if DROP_TABLES:
    drop_table(ERC721_ANALYSIS_DISTRIBUTION_TABLE_NAME, dataset_name=ERC721_ANALYSIS_DATASET_NAME)

if CREATE_TABLES:
    create_table(query_2, ERC721_ANALYSIS_DISTRIBUTION_TABLE_NAME, dataset_name=ERC721_ANALYSIS_DATASET_NAME)

Table erc721_analysis:erc721_analysis_distribution has been deleted.
Table erc721_analysis:erc721_analysis_distribution has been created and filled 51776 rows.
